In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

# Load the crime data
crime_data_path = 'Crime_Data_from_2020_to_Present.csv'
crime_data = pd.read_csv(crime_data_path)

# Convert 'DATE OCC' to datetime to facilitate filtering by year
crime_data['DATE OCC'] = pd.to_datetime(crime_data['DATE OCC'])

# Load the LA County GeoJSON as a GeoDataFrame
la_regions_geojson_path = 'los-angeles-county.geojson'
la_regions = gpd.read_file(la_regions_geojson_path)

# Convert crime data to a GeoDataFrame
crime_gdf = gpd.GeoDataFrame(crime_data, geometry=gpd.points_from_xy(crime_data['LON'], crime_data['LAT']))

# Ensure both GeoDataFrames use the same CRS for accurate spatial operations
crime_gdf.crs = la_regions.crs

# Filter theft-related crimes for demonstration purposes
theft_crimes = crime_gdf[crime_gdf['Crm Cd Desc'].str.contains('THEFT', case=False, na=False)]

# Spatial join to associate crimes with regions
joined_data = gpd.sjoin(theft_crimes, la_regions, how='inner', op='intersects')

# Aggregate crime data by region
crime_stats = joined_data.groupby('name').size().reset_index(name='crime_count')

# Merge aggregated crime data back with LA regions GeoDataFrame
la_regions_stats = la_regions.merge(crime_stats, on='name', how='left').fillna(0)

# Create a base map
m = folium.Map(location=[34.0522, -118.2437], zoom_start=11)

# Add a choropleth layer to visualize theft crime statistics by region
choropleth = folium.Choropleth(
    geo_data=la_regions_stats.to_json(),
    name='Theft Crime Statistics by Region',
    data=la_regions_stats,
    columns=['name', 'crime_count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Theft Crimes'
).add_to(m)

# Add tooltips to the choropleth layer
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['name', 'crime_count'], aliases=['Region:', 'Theft Crimes:'])
)

# Create MarkerClusters for each year of interest
years = [2020, 2021,2022,2023]
for year in years:
    year_filtered_data = crime_data[(crime_data['DATE OCC'].dt.year == year) & 
                                    crime_data['Crm Cd Desc'].str.contains('THEFT')]
    marker_cluster = MarkerCluster(name=f'Theft Crimes {year}')
    for index, row in year_filtered_data.iterrows():
        folium.Marker(
            location=[row['LAT'], row['LON']],
            popup=f"{row['Crm Cd Desc']} on {row['DATE OCC'].date()}"
        ).add_to(marker_cluster)
    m.add_child(marker_cluster)

# Add layer control to switch between layers
folium.LayerControl().add_to(m)

# Save the map to an HTML file

m.save('la_theft_crime_map.html')